In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from transformers import pipeline
from sentence_transformers import SentenceTransformer



embedder = pipeline(
    "feature-extraction",
    model="bert-base-uncased",
    tokenizer="bert-base-uncased",
    device=0   
)



Device set to use mps:0


In [2]:
train_df = pd.read_csv("data/kaggle/train.csv")

label_cols = [
    "toxic","severe_toxic","obscene",
    "threat","insult","identity_hate"
]

In [3]:
# 2) Text cleaning (same as BoW)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"https?://\S+", " ", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

train_df["comment_text"] = train_df["comment_text"].apply(clean_text)


In [4]:
# 3) Train/Validation split
X = train_df["comment_text"].to_numpy()
y = train_df[label_cols].to_numpy()
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.1,
    random_state=42
)



In [5]:
# 4) BERT feature-extraction pipeline
#    Here we use the CPU if no GPU; if you have GPU, set device=0
embedder = pipeline(
    "feature-extraction",
    model="bert-base-uncased",
    tokenizer="bert-base-uncased",
    device=0   # change to -1 for CPU only
)

def get_cls_embeddings(texts, batch_size=8):
    """Return a NumPy array of shape (len(texts), hidden_size)."""
    embs = embedder(texts, batch_size=batch_size, truncation=True)
    # each entry embs[i] is shape (seq_len, hidden_size); take the [CLS] token at pos 0
    cls = [e[0] for e in embs]
    return np.vstack(cls)

Device set to use mps:0


In [7]:

model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')  

X_train_emb = model.encode(
    X_train, 
    batch_size=64, 
    show_progress_bar=True, 
    convert_to_numpy=True
)
X_val_emb = model.encode(
    X_val, 
    batch_size=64, 
    show_progress_bar=True, 
    convert_to_numpy=True
)

print("Train embeddings shape:", X_train_emb.shape)
print("Val embeddings   shape:", X_val_emb.shape)

Batches:   0%|          | 0/2244 [00:00<?, ?it/s]

KeyboardInterrupt: 